In [127]:
# https://www.its.go.kr/opendata/opendataList?service=event
import folium
import pandas as pd

# 126.377157 129.426225
# 33.474579 38.1947

file_name = "[EVENT]2024-11-29.csv"
df = pd.read_csv(file_name)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   돌발일시    1941 non-null   object 
 1   관리기관    1941 non-null   object 
 2   도로타입    1941 non-null   object 
 3   도로번호    1941 non-null   object 
 4   도로명     1941 non-null   object 
 5   방향      1941 non-null   object 
 6   돌발구분    1941 non-null   object 
 7   돌발상세구분  1941 non-null   object 
 8   돌발내용    1900 non-null   object 
 9   X       1941 non-null   float64
 10  Y       1941 non-null   float64
 11  링크아이디   1941 non-null   object 
 12  돌발종료일시  1941 non-null   object 
dtypes: float64(2), object(11)
memory usage: 197.3+ KB


In [128]:
df.drop(
    ["링크아이디", "관리기관", "돌발일시", "돌발종료일시", "도로타입", "돌발상세구분"],
    axis=1,
    inplace=True,
)

# 지도 생성 및 현재 위치 표시
nowX = 126.9180
nowY = 37.5165
mymap = folium.Map(
    location=[nowY, nowX], tiles="cartodbpositron", zoom_start=14, width=600, height=600
)

# 현재 위치에 마커 추가
folium.Marker(
    [nowY, nowX],
    icon=folium.DivIcon(html='<div style="font-size:35px;">🚘</div>'),
    popup="My Location",
).add_to(mymap)


# df에 존재하는 모든 좌표에 작은 원 추가
for idx, row in df.iterrows():
    if row["돌발구분"] == "공사":
        folium.Marker(
            location=[row["Y"], row["X"]],
            icon=folium.DivIcon(html='<div style="font-size:24px;">🚧</div>'),
        ).add_to(mymap)
    elif row["돌발구분"] == "결빙":
        folium.Marker(
            location=[row["Y"], row["X"]],
            icon=folium.DivIcon(html='<div style="font-size:24px;">🧊</div>'),
        ).add_to(mymap)
    elif row["돌발구분"] == "교통사고":
        folium.Marker(
            location=[row["Y"], row["X"]],
            icon=folium.DivIcon(html='<div style="font-size:24px;">🚨</div>'),
        ).add_to(mymap)
    else:
        folium.Marker(
            location=[row["Y"], row["X"]],
            icon=folium.DivIcon(html='<div style="font-size:24px;">⚠</div>'),
        ).add_to(mymap)

mymap

In [129]:
# df에서 현재 위치와 0.01 이내인 돌발사고만 추출
df = df[(df["Y"] - nowY).abs() <= 0.01]
df = df[(df["X"] - nowX).abs() <= 0.01]

for idx, row in df.iterrows():
    print(row["돌발내용"])

<공사> 올림픽대로 샛강IC 대방동4981 1차로 [공사] 2024년 자동차전용도로 위험노목 정비공사 / 장소: 올림픽대로 (여의2교~여의교) 잠실방향 / 부분통제 도로공사  서울청
<사고> 경인로 서울교 영등포로터리 5차로  교통사고  서울청
<사고> 국회대로 영동초등학교교차로  여의2교교차로  4차로  교통사고  서울청 
<사고> 경인로 서울교 영등포로터리 2차로  교통사고  서울청
<사고> 경인로 서울교 영등포로터리 6차로  교통사고  서울청
